### Fama-French-Carhart model (four factor model)

The CAPM considers that the return of a stock is mainly influenced by the market return.  
The Fama French Model, also known as the three-factor model, complements the CAPM as it assumes not one, but three major factors influencing the return of a financial asset: the market return, the size of the company (SMB), and its book-to-market ratio (HML).  
This model can be extended with the four-factor model (Fama-French-Carhart model) that adds the momentum effect (WML).  
They define an equation to link these elements to the return of the asset under study:

$$Ri = rf + ß1 (Rm - rf) + ß2(SMB) + ß3(HML) + ß4(WML) + ε$$

To do this, this code will form each of the portfolios by updating the data each time (While these portfolios are usually calculated once a year). Moreover, you can form the portfolios with the sample of companies you want.

**Ri**: return of the stock i  
**rf**: risk-free rate  
**ß**: measure of the volatility of the stock price relative to the portfolio (ß1 for the market, ß2 for the SMB portfolio...)  
$ß = Cov(Ri, R) / Var(R)$ with R the return of the portfolio (either the return of the market, of the SMB portfolio...)

We label the portfolios as follows:
SG: Small-Growth, SN: Small-Neutral, SV: Small-Value,
BG: Big-Growth, BN: Big-Neutral, BV: Big-Value,
SW: Small-Winners, SL: Small-Losers,
BW: Big-Winners, BL: Big-Losers.

**SMB**: this is the Small-Minus-Big portfolio. It has been observed that small companies tend to outperform big companies.  
Breakpoint: The 80th percentile of the market capitalization.  
While companies above the 80th percentile are marked as Big, companies below this threshold are marked as Small.  

The size (SMB) factor is the average return on the 3 small portfolios minus the average return on the 3 big portfolios:  
$$SMB = (SG + SN + SV) / 3 – (BG + BN + BV) / 3$$

**HML**: this is the High-Minus-Low portfolio. It has been observed that companies with a high BM (Book-to-Market) ratio (Value stocks) tend to outperform companies with a lower ratio (growth stocks).  
Breakpoints: The 30th and 70th percentiles of the Book-to-Market ratio.  
While companies above the 70th percentile are marked as Value, companies below the 30th percentile are marked as Growth.  

The value (HML) is the average return on the 3 value portfolios minus the average return on the 2 big growth:  
$$HML = (SV + BV) / 2 – (SG + BG) / 2$$

**WML**: this is the Winner-Minus-Loser portfolio. It measures the momentum effect. It has been observed that companies with bullish stock prices tend to outperform companies with bearish stock prices.  
Breakpoints: 10th and 90th percentiles.  
While companies above the 90 percentile are marked as Winner, companies below the 10 percentile are marked as Loser.  

Note: we sort the companies based on their one-year return. However, we remove the last month, which should make it a return over 11 months. Why? There is a short-term reversal in stocks, so we remove the last month to avoid mixing the momentum effect with that of the reversal.

The momentum (WML) is the average return on the 2 high return portfolios minus the average return on the 2 low return portfolios:  
$$WML = (SW + BW) / 2 – (SL + BL) / 2$$

Source: [https://www.hhs.se/globalassets/swedish-house-of-finance/data-center/fama_french_methodology.pdf](https://www.hhs.se/globalassets/swedish-house-of-finance/data-center/fama_french_methodology.pdf)

### Rules for the inputs : 

**rf** : Define the risk-free rate (default 3%)    
**arg** : Define the study period in years (default 5 years) and the Investment horizon interval: Valid intervals: [1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]    
**market** : Define the benchmark (default is CAC40)    
**tickers_list** : Define the companies to analyze    
**targeted_company**: See the result for a specific company

In [25]:
from yahooquery import Ticker
import numpy as np
import yfinance as yf
import pandas as pd

In [26]:
tickers_list = ["AI.PA", "AIR.PA", "TTE.PA", "DEC.PA", "ALO.PA", "GLE.PA","MT.AS", "CS.PA", "BNP.PA", "EN.PA", "CAP.PA", "CA.PA", "ACA.PA", "BN.PA", "DSY.PA", "ENGI.PA", "EL.PA", "ERF.PA", "RMS.PA", "KER.PA","LR.PA", "OR.PA", "MC.PA", "ML.PA", "ORA.PA", "RI.PA", "PUB.PA", "RNO.PA", "SAF.PA", "SGO.PA", "SAN.PA", "SU.PA", "STMPA.PA", "TEP.PA", "HO.PA", "VIE.PA", "DG.PA", "WLN.PA"]
targeted_company = "AI.PA"
rf = 0.03
market = "^FCHI"
arg = {"period": "5y", "interval": "1wk"}

In [27]:
## We retrieve the Book-to-Market ratio and the market capitalization of each company
dic_book_market_ratio = {}
dic_market_cap = {}
list_ent_with_data = []
for ent in tickers_list:
    try:
        df = Ticker(ent).valuation_measures
        book_to_market_ratio = 1 / df["PbRatio"].dropna()[-1]
        dic_book_market_ratio[ent] = book_to_market_ratio
        dic_market_cap[ent] = df["MarketCap"].dropna()[-1]
        list_ent_with_data.append(ent)
    except:
        None
tickers_list = list_ent_with_data


In [28]:
## Creation of a general DataFrame that gathers the returns of all the stocks
df = pd.DataFrame()
for ent in tickers_list + [market]:
    stock = yf.Ticker(ent)
    data = stock.history(**arg)
    returns = data["Close"].pct_change().dropna()
    df = pd.concat([df, returns], axis=1)
    df.columns.values[-1] = ent


In [29]:
# Construction Small and Big
sorted_dic_market_cap = dict(sorted(dic_market_cap.items(), key=lambda item: item[1]))
sorted_market_cap = list(sorted_dic_market_cap.values())
percentile_1 = np.percentile(sorted_market_cap, 80)
percentile_3 = np.percentile(sorted_market_cap, 80)
list_big = []
list_small = []

for i in sorted_dic_market_cap:
    if sorted_dic_market_cap[i] <= percentile_1:
        list_small.append(i)
    elif sorted_dic_market_cap[i] >= percentile_3:
        list_big.append(i)

# Construction S/G S/M and S/V
small_dic_book_market_ratio = {key: dic_book_market_ratio[key] for key in list_small}
small_sorted_dic_book_market_ratio = dict(sorted(small_dic_book_market_ratio.items(), key=lambda item: item[1]))
small_sorted_ratio = list(small_sorted_dic_book_market_ratio.values())
percentile_1 = np.percentile(small_sorted_ratio, 30)
percentile_3 = np.percentile(small_sorted_ratio, 70)
list_small_low = []
list_small_high = []
list_small_medium = []

for i in list_small:
    if small_sorted_dic_book_market_ratio[i] <= percentile_1:
        list_small_low.append(i)
    elif small_sorted_dic_book_market_ratio[i] >= percentile_3:
        list_small_high.append(i)
    else:
        list_small_medium.append(i)

df_global = pd.DataFrame()
df_global["S/G"] = df[list_small_low].mean(axis=1)
df_global["S/M"] = df[list_small_medium].mean(axis=1)
df_global["S/V"] = df[list_small_high].mean(axis=1)

# Construction B/G B/M B/V
big_dic_book_market_ratio = {key: dic_book_market_ratio[key] for key in list_big}
big_sorted_dic_book_market_ratio = dict(sorted(big_dic_book_market_ratio.items(), key=lambda item: item[1]))
big_sorted_ratio = list(big_sorted_dic_book_market_ratio.values())
percentile_1 = np.percentile(big_sorted_ratio, 30)
percentile_3 = np.percentile(big_sorted_ratio, 70)
list_big_low = []
list_big_high = []
list_big_medium = []

for i in list_big:
    if big_sorted_dic_book_market_ratio[i] <= percentile_1:
        list_big_low.append(i)
    elif big_sorted_dic_book_market_ratio[i] >= percentile_3:
        list_big_high.append(i)
    else:
        list_big_medium.append(i)

df_global["B/G"] = df[list_big_low].mean(axis=1)
df_global["B/M"] = df[list_big_medium].mean(axis=1)
df_global["B/V"] = df[list_big_high].mean(axis=1)

# Construction S/L and S/W
small_dic_past_returns = {key: df[key][-52:-1].cumsum().iloc[-1] for key in list_small}
small_sorted_dic_past_returns = dict(sorted(small_dic_past_returns.items(), key=lambda item: item[1]))
small_sorted_past_returns = list(small_sorted_dic_past_returns.values())
percentile_1 = np.percentile(small_sorted_past_returns, 10)
percentile_3 = np.percentile(small_sorted_past_returns, 90)
list_small_winners = []
list_small_losers = []

for i in small_sorted_dic_past_returns:
    if small_sorted_dic_past_returns[i] <= percentile_1:
        list_small_losers.append(i)
    elif small_sorted_dic_past_returns[i] >= percentile_3:
        list_small_winners.append(i)

df_global["S/L"] = df[list_small_losers].mean(axis=1)
df_global["S/W"] = df[list_small_winners].mean(axis=1)

# Construction B/L and B/W
big_dic_past_returns = {key: df[key][-52:-1].cumsum().iloc[-1] for key in list_big}
big_sorted_dic_past_returns = dict(sorted(big_dic_past_returns.items(), key=lambda item: item[1]))
big_sorted_past_returns = list(big_sorted_dic_past_returns.values())
percentile_1 = np.percentile(big_sorted_past_returns, 10)
percentile_3 = np.percentile(big_sorted_past_returns, 90)
list_big_winners = []
list_big_losers = []

for i in big_sorted_dic_past_returns:
    if big_sorted_dic_past_returns[i] <= percentile_1:
        list_big_losers.append(i)
    elif big_sorted_dic_past_returns[i] >= percentile_3:
        list_big_winners.append(i)

df_global["B/L"] = df[list_big_losers].mean(axis=1)
df_global["B/W"] = df[list_big_winners].mean(axis=1)

# Construction small stock portfolio
df_global["Small"] = df_global[["S/G", "S/M", "S/V"]].mean(axis=1)

# Construction big stock portfolio
df_global["Big"] = df_global[["B/G", "B/M", "B/V"]].mean(axis=1)

# Construction value stock portfolio
df_global["Value"] = df_global[["S/V", "B/V"]].mean(axis=1)

# Construction growth stock portfolio
df_global["Growth"] = df_global[["S/G", "B/G"]].mean(axis=1)

# Construction Winners
df_global["Winners"] = df_global[["S/W", "B/W"]].mean(axis=1)

# Construction Losers
df_global["Losers"] = df_global[["S/L", "B/L"]].mean(axis=1)

# Construction SMB portfolio
df_global["SMB"] = df_global["Small"] - df_global["Big"]

# Construction HML portfolio
df_global["HML"] = df_global["Value"] - df_global["Growth"]

# Construction WML portfolio
df_global["WML"] = df_global["Winners"] - df_global["Losers"]

df_global


,S/G,S/M,S/V,B/G,B/M,B/V,S/L,S/W,B/L,B/W,Small,Big,Value,Growth,Winners,Losers,SMB,HML,WML
2018-09-10 00:00:00+02:00,0.009243,0.013708,0.024418,0.015881,0.045902,0.026662,0.031361,0.022262,0.028858,0.021116,0.015790,0.029482,0.025540,0.012562,0.021689,0.030109,-0.013692,0.012978,-0.008420
2018-09-17 00:00:00+02:00,0.008326,0.017167,0.035173,0.020142,0.036799,0.013188,-0.023531,0.058653,0.046177,0.033973,0.020222,0.023376,0.024180,0.014234,0.046313,0.011323,-0.003154,0.009946,0.034990
2018-09-24 00:00:00+02:00,0.000061,0.001355,-0.014495,0.013204,-0.010136,0.023991,0.028921,-0.038675,-0.004087,0.018929,-0.004360,0.009020,0.004748,0.006633,-0.009873,0.012417,-0.013379,-0.001885,-0.022290
2018-10-01 00:00:00+02:00,-0.023916,-0.017540,-0.023117,-0.043787,-0.052714,-0.007122,-0.070898,-0.015105,-0.060407,-0.062390,-0.021524,-0.034541,-0.015119,-0.033852,-0.038748,-0.065653,0.013017,0.018732,0.026905
2018-10-08 00:00:00+02:00,-0.065086,-0.053958,-0.036759,-0.067577,-0.073892,-0.043143,-0.082970,-0.043717,-0.082809,-0.080374,-0.051934,-0.061537,-0.039951,-0.066331,-0.062046,-0.082890,0.009603,0.026380,0.020844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-11 00:00:00+02:00,-0.004789,0.025151,0.029627,0.004370,0.015343,0.024355,0.002054,0.036785,0.028649,0.029307,0.016663,0.014689,0.026991,-0.000210,0.033046,0.015351,0.001974,0.027201,0.017694
2023-09-18 00:00:00+02:00,-0.029306,-0.022986,-0.031646,-0.047724,-0.029463,-0.019644,-0.020039,-0.019775,-0.037966,-0.058100,-0.027979,-0.032277,-0.025645,-0.038515,-0.038938,-0.029003,0.004297,0.012870,-0.009935
2023-09-25 00:00:00+02:00,-0.004004,-0.015003,-0.005942,-0.007717,-0.001644,0.002661,-0.023733,0.008259,-0.021712,-0.035248,-0.008316,-0.002233,-0.001640,-0.005861,-0.013495,-0.022723,-0.006083,0.004221,0.009228
2023-10-02 00:00:00+02:00,-0.003569,-0.018083,-0.068866,-0.001114,0.004823,-0.012345,-0.030903,-0.017005,0.018565,0.015377,-0.030173,-0.002878,-0.040605,-0.002341,-0.000814,-0.006169,-0.027294,-0.038264,0.005356


In [30]:
## Application of the Fama French Carhart Model to all the companies
liste = tickers_list

list_expected_return = []
for stock_to_analyze in liste:
    return_stock = df[stock_to_analyze]
    
    return_market = df[market]

    beta_market = np.cov(return_stock, return_market)[0, 1] / return_market.var()
    first_factor = beta_market * (return_market.mean() - rf)

    beta_HML = np.cov(return_stock, df_global["HML"])[0, 1] / df_global["HML"].var()
    second_factor = beta_HML * df_global["HML"].mean()

    beta_SMB = np.cov(return_stock, df_global["SMB"])[0, 1] / df_global["SMB"].var()
    third_factor = beta_SMB * df_global["SMB"].mean()
    
    beta_WML = np.cov(return_stock, df_global["WML"])[0, 1] / df_global["WML"].var()
    fourth_factor = beta_WML * df_global["WML"].mean()

    expected_return = rf + first_factor + second_factor + third_factor + fourth_factor
    list_expected_return.append(expected_return)

df_final = pd.DataFrame({'Company': liste, 'Expected return': list_expected_return})
df_final.sort_values(by="Expected return")


,Company,Expected return
5,GLE.PA,-0.021386
27,RNO.PA,-0.019051
6,MT.AS,-0.015814
1,AIR.PA,-0.014384
8,BNP.PA,-0.012344
28,SAF.PA,-0.011732
12,ACA.PA,-0.010884
29,SGO.PA,-0.009061
32,STMPA.PA,-0.007908
7,CS.PA,-0.006379


In [40]:
max_with_outliers = df_final.loc[df_final['Expected return'].idxmax()]
print("Max with outliers :\n", max_with_outliers[0], round(max_with_outliers[-1], 5))
print()

# We implement a filter to eliminate any potential outliers
filter_on_value = (df_final['Expected return'] >= -0.2) & (df_final['Expected return'] <= 0.2)
df_without_outliers = df_final.mask(~filter_on_value).dropna()
max_without_outliers = df_without_outliers.loc[df_without_outliers['Expected return'].idxmax()]
print("Max without outliers :\n", max_without_outliers[0], round(max_without_outliers[-1], 5))


Max with outliers :
 SAN.PA 0.01673

Max without outliers :
 SAN.PA 0.01673


In [42]:
# Look at the model for a specific company
ligne_x = df_final[df_final['Company'] == targeted_company]
specific_expected_return = ligne_x['Expected return'].iloc[0]
print("Expected return of", targeted_company, ":\n", round(specific_expected_return, 5))

Expected return of AI.PA :
 0.01078
